# Final results

In [1]:
import pandas as pd
import pickle

In [6]:
test_df = pd.read_csv('../data/BipolarDerivedDataTesting.csv')
test_df.head()

,record_id,Age,Sex,Hand,allvol_unknown,allvol_ctx_rh_posterior_insula,allvol_ctx_lh_posterior_insula,allvol_ctx_rh_parietal_operculum,allvol_ctx_lh_parietal_operculum,allvol_left_lateral_ventricle,...,31p_cc_ant_gatp,31p_cc_ant_gatpr,31p_cc_ant_aatp,31p_cc_ant_aatpr,31p_l_pat_ph,31p_l_pat_pcr,31p_l_pat_gatp,31p_l_pat_gatpr,31p_l_pat_aatp,31p_l_pat_aatpr
0,1188,62,Female,Right,5567321,1912,2069,2248,2383,180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1248,52,Female,Right,5877548,2468,2095,2636,2531,229,...,7791240.0,0.552266,3807530.0,0.311396,7.34786,7500230.0,6585610.0,0.538620,4439060.0,0.554594
2,1197,52,Female,Ambidextrous,6134859,1583,1601,1921,2591,252,...,7091470.0,0.933621,4043440.0,0.333823,7.14109,16732700.0,6694770.0,0.737140,5465680.0,0.367607
3,1608,27,Male,Right,6696095,3057,2857,2861,3154,251,...,5204750.0,0.494524,3032390.0,0.218201,7.15694,14720600.0,6039600.0,0.425295,3960720.0,0.275689
4,1987,26,Female,Left,5894652,2345,1956,2323,2397,191,...,12149900.0,1.179160,3914360.0,0.595178,7.23300,9042540.0,12212100.0,2.173140,3694640.0,0.656156


### Import Kerry's model for bp prediction

In [7]:
with open('../models/kerry-pipeline1_prediction.pkl', 'rb') as f:
    search = pickle.load(f)
kerry_test_df = test_df.filter(regex=("t1r_.*m"))
pred_arr = search.predict(kerry_test_df)
pred_arr

array(['Case', 'Control', 'Case', 'Case', 'Control', 'Control', 'Control',
       'Case', 'Case', 'Control', 'Case', 'Control', 'Case', 'Case',
       'Case', 'Case', 'Case', 'Control', 'Case', 'Case', 'Case', 'Case',
       'Case', 'Case', 'Case', 'Case', 'Control', 'Control', 'Control',
       'Control', 'Control', 'Case', 'Case', 'Case', 'Case', 'Case',
       'Case', 'Case', 'Case'], dtype=object)

In [11]:
kerry_df = pd.DataFrame()
case_control = [1 if pred == 'Case' else 0 for pred in list(pred_arr)]
case_control.count(1)/len(case_control)

0.6923076923076923

In [14]:
kerry_df['ID'] = test_df['record_id']
kerry_df['case_control'] = case_control
kerry_df.to_csv('../data/kerry-bp_hackathon_submission.txt', index=False)

### Import James' model for bp prediction